In [2]:
 #!pip install gensim

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
    100% |████████████████████████████████| 24.7MB 952kB/s eta 0:00:01
    100% |████████████████████████████████| 27.4MB 881kB/s eta 0:00:01
    100% |████████████████████████████████| 13.9MB 1.9MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 5.2MB/s ta 0:00:011
    100% |████████████████████████████████| 1.4MB 6.1MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 4.3MB/s ta 0:00:011
    100% |████████████████████████████████| 133kB 18.8MB/s ta 0:00:01
    100% |████████████████████████████████| 153kB 15.3MB/s ta 0:00:01
    100% |████████████████████████████████| 163kB 17.7MB/s ta 0:00:01
    100% |████████████████████████████████| 143kB 18.3MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 23.2MB/s ta 0:00:01
    100% |███████

In [149]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import pandas as pd
import gensim
import nltk
import json
import re

In [20]:
"""
from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
all credits go to alko and arturomp @ stack overflow.
"""

f = open('../Data_Story/wordLists/contractionList.txt', 'r')
cList = json.loads(f.read())
c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [437]:
#from nltk.corpus import stopwords
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')
#stop_words = ['do','got','did','made', 'make','also','been','like','is', 'was', 'were', \
#               'are', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', \
#               "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', \
#               'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', \
#               "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', \
#               'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'a', \
#               'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
#               'at', 'by', 'for', 'with', 'about', 'against', 'between', 'through', 'during', \
#               'before', 'after', 'above', 'below', 'to', 'from', 'in', 'out', 'on', 'off', \
#               'under', 'again', 'further', 'then', 'once', 'here', 'there', 'why', 'how', \
#               'all', 'any', 'both', 'each', 'other', 'some', 'such', 'same', 'so', 'than', \
#               'too', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", \
#               'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', \
#               "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', \
#               "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", \
#               'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", \
#               'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't", 'yeah', \
#               'yea', 'san', 'fran', 'francisco']

new_words=('yeah', 'yea', 'also', 'get', 'got', 'etc','theyvenconcocted', 'ive', 'san', 'fran', \
           'francisco', 'sf', 'finally', 'wasnt', 'cool', 'end', 'im' , 'yelp', 'could', 'would', \
           'cannot', 'wouldnt','way', 'day', 'yum', )
for i in new_words:
    stop_words.append(i)

In [438]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 
    
def normalizer(paragrahp):
    paragrahp = expandContractions(paragrahp.lower())
    paragrahp_c = paragrahp.split('.')[:-1]
    # lower case and remove special character/whitespace
    paragrahp = [re.sub(r'[^a-zA-Z\s]','', sentence) for sentence in paragrahp_c]
    #paragrahp = [sentence.strip() for sentence in paragrahp]
    tokens = [wpt.tokenize(sentence) for sentence in paragrahp]
    tokens_filtered = [[word for word in token if word not in stop_words] for token in tokens]
    tokens_lemmatizer = [[lemmatizer.lemmatize(word) for word in token] for token in tokens_filtered]
    #paragrahp_norm = [' '.join(tokens) for tokens in tokens_filtered]
    return tokens_lemmatizer

In [439]:
df = pd.read_csv('../Data_Extraction/Reviews/reviews_yelp_scrapping_rating.csv')

In [440]:
df.head()

,Name,Description,Rating
0,Native Twins Coffee,"[""This place to caters to all types of eaters....","['5.0 star rating', '3.0 star rating', '5.0 st..."
1,Home,"[""Another cafe that has been on my bookmark in...","['5.0 star rating', '3.0 star rating', '5.0 st..."
2,Home,"[""Visiting friends in SF, my husband was cravi...","['5.0 star rating', '4.0 star rating', '4.0 st..."
3,Réveille Coffee Co.,"[""Love the vibe in this cozy little space! It'...","['5.0 star rating', '1.0 star rating', '4.0 st..."
4,Neighbor’s Corner,"[""This charming cafe served up much more than ...","['5.0 star rating', '5.0 star rating', '5.0 st..."


In [458]:
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

def lda_coffeeshop(id, topics=1, num_words=1):
    data = df['Description'][id]
    texts = normalizer(data)
    #print (texts)
    
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=topics, id2word = dictionary, passes=50, random_state=10)
    a = ldamodel.print_topics(num_topics=topics, num_words=num_words)
    print('\nPerplexity: ', ldamodel.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=ldamodel, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence_lda)
    return a

In [462]:
lda = lda_coffeeshop(0, 6, 20)


Perplexity:  -6.697483336300947

Coherence Score:  0.45592384126980434

Perplexity:  -6.138733846082285

Coherence Score:  0.44902192790552836


In [456]:
#lda1 = lda_coffeeshop(0, 1, 100)


In [463]:
lda1

[(0,
  '0.025*"coffee" + 0.013*"latte" + 0.012*"place" + 0.012*"toast" + 0.010*"good" + 0.010*"great" + 0.010*"twin" + 0.009*"shop" + 0.009*"milk" + 0.006*"perfect" + 0.006*"really" + 0.006*"always" + 0.006*"delicious" + 0.006*"avocado" + 0.006*"matcha" + 0.006*"spot" + 0.005*"back" + 0.005*"native" + 0.005*"love" + 0.005*"super" + 0.005*"like" + 0.004*"nice" + 0.004*"though" + 0.004*"one" + 0.004*"turmeric" + 0.004*"bit" + 0.004*"cute" + 0.004*"favorite" + 0.004*"owned" + 0.004*"plus" + 0.004*"stopped" + 0.004*"cold" + 0.004*"friendly" + 0.004*"pastry" + 0.004*"brew" + 0.003*"definitely" + 0.003*"local" + 0.003*"bread" + 0.003*"service" + 0.003*"small" + 0.003*"drink" + 0.003*"business" + 0.003*"coming" + 0.003*"side" + 0.003*"tried" + 0.003*"chai" + 0.003*"made" + 0.003*"haight" + 0.003*"discovered" + 0.003*"enjoyed" + 0.003*"kind" + 0.003*"morning" + 0.003*"sister" + 0.003*"lovely" + 0.003*"pretty" + 0.003*"amazing" + 0.003*"start" + 0.003*"neighborhood" + 0.003*"everything" + 0.003

In [489]:
data = df['Description'][1]
paragrahp = expandContractions(data).lower().split('.')
paragrahp = [re.sub(r'[^a-zA-Z\s]','', sentence) for sentence in paragrahp]
tokens = [wpt.tokenize(sentence) for sentence in paragrahp]
tokens_lemmatizer = [[lemmatizer.lemmatize(word) for word in token] for token in tokens]
tokens_filtered = [[word for word in sentence if word not in stop_words] for sentence in tokens_lemmatizer]
#print (tokens_filtered)


In [490]:
a = [['coffee', 'toast', 'avocado', 'bean', 'barista', 'neghborhood', \
          'tiny', 'small', 'wifi', 'tasty', 'little', 'good', 'perfect', \
         'latte', 'capuccino', 'matcha', 'coming', 'cafe', 'pretty', 'almond', \
          'amazing']]
dictionary = corpora.Dictionary(a)

In [491]:
corpus = [dictionary.doc2bow(text) for text in tokens_filtered]
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=1000)
ldamodel.print_topics(num_topics=3, num_words=5)

[(0,
  '0.315*"pretty" + 0.264*"little" + 0.213*"cafe" + 0.060*"wifi" + 0.010*"good"'),
 (1,
  '0.432*"latte" + 0.267*"toast" + 0.179*"good" + 0.026*"avocado" + 0.025*"tasty"'),
 (2,
  '0.552*"coffee" + 0.123*"perfect" + 0.094*"coming" + 0.066*"small" + 0.011*"latte"')]

In [195]:
list_rating = df['Rating']

In [200]:
r = [j.split('.')[0][-1] for ratings in list_rating for j in ratings.split(',')]

In [ ]:
df['score'] = df['Rating'].str.split(' star rating').str.get(0)